In [50]:
import pandas as pd
import json
import re

Funções de Validação

In [55]:
def validacao_df_importacao(arquivo_excel):
    df = pd.read_excel( f'Arquivos/{arquivo_excel}.xlsx', sheet_name='lancamentos' )
    # Manipulação das Colunas e dos dados
    df['procedencia'] = df['procedencia'].astype( str )  # Define como string
    df['procedencia'] = df['procedencia'].str.strip()  # Retira os espaços de ambos os lados
    df['procedencia'] = df['procedencia'].apply( lambda x: x.upper() )  # Torna todas as letras maiúsculas

    df['destino'] = df['destino'].astype( str )  # Define como string
    df['destino'] = df['destino'].str.strip()  # Retira os espaços de ambos os lados
    df['destino'] = df['destino'].apply( lambda x: x.upper() )  # Torna todas as letras maiúsculas

    df['produto'] = df['produto'].astype( str )  # Define como string
    df['produto'] = df['produto'].str.strip()  # Retira os espaços de ambos os lados
    df['produto'] = df['produto'].apply( lambda x: x.upper() )  # Torna todas as letras maiúsculas
    df['produto'] = df['produto'].apply( lambda x: x.replace( '\\', '/' ) )


    # Abre o arquivo Json de procedências e salva em uma variável
    with open( 'dicionario_procedencia.json', 'r', encoding='utf-8' ) as arquivo_json:
        dicionario_procedencia = json.load( arquivo_json )
    for cidade, valor in dicionario_procedencia.items():
        df.loc[df['procedencia'] == cidade, 'procedencia'] = valor
    # ----------------------------------------------------------------------------
    # Abre o arquivo Json de produtos e salva em uma variável
    with open( 'dicionario_produtos.json', 'r', encoding='utf-8' ) as arquivo_produtos_json:
        dicionario_produtos = json.load( arquivo_produtos_json )
    for produto, valor in dicionario_produtos.items():
        df.loc[df['produto'] == produto, 'produto'] = valor

    # Abre o arquivo Json de produtos e salva em uma variável
    with open( 'dicionario_destinos.json', 'r', encoding='utf-8' ) as arquivo_destino_json:
        dicionario_destino = json.load( arquivo_destino_json )
    for destino, valor in dicionario_produtos.items():
        df.loc[df['destino'] == destino, 'destino'] = valor

    return df

def lista_cidades_com_erros(df):
    '''
        Verifica se alguma cidade da coluna procedência não está na lista de procedências
    '''
    df_verificacao = df['procedencia'].loc[~df['procedencia'].str.contains( r'(', regex=False )].unique()
    lista_erros_cidades = []
    if df_verificacao.size == 0:
        return print( 'Tudo certo por aqui!' )
    else:
        lista = list( df['procedencia'].loc[~df['procedencia'].str.contains( r'(', regex=False )].unique() ) 
        for elemento in lista:
            lista_erros_cidades.append(elemento)
        return lista_erros_cidades

def lista_produtos_com_erros(df):
    '''
        Verifica se algum produto não está na listagem padronizada
    '''
    lista_produtos = []
    lista_para_ajustes = []

    with open( 'lista_produtos.txt', 'r' ) as lista_txt:
        for item in lista_txt:
            texto = item.strip()  # Remove newline characters
            lista_produtos.append( texto )

    for item in df['produto']:
        if item not in lista_produtos:
            lista_para_ajustes.append( item )

    lista_para_ajustes = set( lista_para_ajustes )
    lista_para_ajustes = list( lista_para_ajustes )
    if len( lista_para_ajustes ) == 0:
        return print( "Lista vazia, parabéns!" )
    else:
        return print( lista_para_ajustes )

def lista_destinos_com_erros(df):
    df['destino'] = df['destino'].apply( lambda x: re.sub( "'", " ", x ) )
    df['atac_gnp'] = df['destino'].apply(lambda x: 'GNP' if x.startswith('01') else 'ATACADISTA' )
    df['atac_gnp'] = df['destino'].apply(lambda x: 'ATACADISTA' if x.startswith('5') else 'GNP' )
    return print( 'Todos os apóstrofos foram retirados e os dados foram separados entre Atacadistas ou '
                  'Produtores' )

def gera_excel_tratado(df):
    df['data'] = df['data'].astype( str )
    df.to_excel( 'Arquivos/dados_tratados.xlsx', index=False )
    return print( f'Arquivo gerado com sucesso!' )

def volume_produtos(dataframe):
    pass

Input dos dados

In [56]:
dataframe = validacao_df_importacao( 'dados_input' )

Validando as procedências

In [57]:
lista_cidades_com_erros( dataframe )

Tudo certo por aqui!


Validando os Produtos

In [58]:
lista_produtos_com_erros( dataframe )

Lista vazia, parabéns!


Validando os nomes dos Destinatários das Mercadorias

In [59]:
lista_destinos_com_erros( dataframe )

Todos os apóstrofos foram retirados e os dados foram separados entre Atacadistas ou Produtores


Inserindo dados de peso e criando a coluna de Volume Comercializado

In [60]:
dataframe.head()

,data,nota_fiscal,procedencia,destino,matricula,produto,unidade,quantidade,atac_gnp
0,2024-01-02,272762,0143128-SAO SEBASTIAO DO CAI (RS),5500358-COM.DE LARANJAS VALE DO,901805890001,00101-ABACATE,0,36,ATACADISTA
1,2024-01-02,39306,0143278-HARMONIA (RS),5501008-COMERCIO DE FRUTAS METZ,11213060001,00101-ABACATE,0,90,ATACADISTA
2,2024-01-02,769016,0143045-FARROUPILHA (RS),5500022-SILVESTRIN IMPORTADORA D,944963530002,00101-ABACATE,0,100,ATACADISTA
3,2024-01-02,39303,0143278-HARMONIA (RS),5501008-COMERCIO DE FRUTAS METZ,11213060001,00101-ABACATE,0,108,ATACADISTA
4,2024-01-02,39305,0143278-HARMONIA (RS),5501008-COMERCIO DE FRUTAS METZ,11213060001,00101-ABACATE,0,180,ATACADISTA


Exportando os dados

In [61]:
gera_excel_tratado( dataframe )

Arquivo gerado com sucesso!
